<center><img src="../logo.png" alt="Header" style="width: 800px;"/></center>

# 颜色识别  Color Recognition

In [1]:
# 中文开关，默认为英文 Chinese switch. The default value is English
g_ENABLE_CHINESE = False

Name_widgets = {
    'Close_Camera': ("Close_Camera", "关闭摄像头")
}

In [2]:
# 显示摄像头组件 Camera display widgets
import ipywidgets.widgets as widgets
from IPython.display import display

import cv2
import time
import numpy as np


import threading

import HSV_Config

In [3]:
#bgr8转jpeg格式
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [4]:
from SunriseRobotLib import Mipi_Camera

img_width=320
img_height=240

g_camera = Mipi_Camera(img_width, img_height)

if g_camera.isOpened():
    print("Open Camera OK")
else:
    print("Fail To Open Camera")


Camera 0:
	i2c_bus: 6
	mipi_host: 0
Camera 1:
	i2c_bus: 4
	mipi_host: 2
Camera 2:
	i2c_bus: 0
	mipi_host: 0
Camera 3:
	i2c_bus: 0
	mipi_host: 0
mipi mclk is not configed.
Searching camera sensor on device: /proc/device-tree/soc/cam/vcon@0 i2c bus: 6 mipi rx phy: 0
INFO: Found sensor name:imx219-30fps on mipi rx csi 0, i2c addr 0x10, config_file:linear_1920x1080_raw10_30fps_2lane.c
2024/12/05 12:09:57.553 !INFO [CamInitParam][0279]Setting VSE channel-0: input_width:1920, input_height:1080, dst_w:320, dst_h:240
Open Camera OK
2024/12/05 12:09:57.553 !INFO [CamInitParam][0279]Setting VSE channel-1: input_width:1920, input_height:1080, dst_w:1920, dst_h:1080
2024/12/05 12:09:57.553 !INFO [vp_vin_init][0041]csi0 ignore mclk ex attr, because mclk is not configed at device tree.


In [5]:
update_hsv = HSV_Config.update_hsv()
# Color_HSV  = {"Red"   : ((0, 43, 46), (10, 255, 255)),
#              "Green" : ((35, 43, 46), (77, 255, 255)),
#              "Blue"  : ((100, 43, 46), (124, 255, 255)),
#              "Yellow": ((26, 43, 46), (34, 255, 255))}
Color_HSV  = {"Red"   : ((0, 111, 67), (16, 255, 255)),
              "Green" : ((43, 90, 65), (90, 255, 255)),
              "Blue"  : ((100, 43, 46), (124, 255, 255)),
              "Yellow": ((17, 138, 106), (34, 255, 255))}
g_stop_program = False

In [6]:
# image_widget = widgets.Image(format='jpeg', width=320, height=240)
# 摄像头显示控件  Camera display widgets
DISPLAY_WIDTH = img_width
DISPLAY_HEIGHT = img_height
image_widget = widgets.Image(format='jpeg', width=DISPLAY_WIDTH, height=DISPLAY_HEIGHT)

In [7]:
# 关闭摄像头 Close_Camera
Button_Close_Camera = widgets.Button(  
    value=False,  
    description=Name_widgets['Close_Camera'][g_ENABLE_CHINESE],      
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''     
    tooltip='Description',     
    icon='uncheck' )


# 按键按下事件处理   Key press event processing
def on_button_close_camera(b):
    global g_stop_program
    if b.description == Name_widgets['Close_Camera'][g_ENABLE_CHINESE]:
        # 停止线程，释放摄像头  Stop the thread and release the camera
        b.icon = 'uncheck'
        g_stop_program = True
        time.sleep(.1)
        g_camera.release()
    
# 关联按键事件回调 Button event callbacks
Button_Close_Camera.on_click(on_button_close_camera)

In [8]:
# 图像处理任务 Image processing tasks
def task_processing():
    global g_stop_program
    t_start = time.time()
    m_fps = 0
    while g_camera.isOpened():
        if g_stop_program:
            break
        ret, frame = g_camera.read()
        frame, binary = update_hsv.get_contours(frame, Color_HSV)
        
        m_fps = m_fps + 1
        fps = m_fps / (time.time() - t_start)
        if (time.time() - t_start) >= 2:
            m_fps = fps
            t_start = time.time() - 1
        cv2.putText(frame, "FPS " + str(int(fps)), (10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 1)
        
        # 图像传输给显示组件 The image is transmitted to the display component
        image_widget.value = bgr8_to_jpeg(frame)
        # time.sleep(.01)
        # print("fps:", int(fps))

In [9]:
# 启动摄像头显示任务  Start the camera display task
thread1 = threading.Thread(target=task_processing)
thread1.daemon = True
thread1.start()


# 创建一个盒子容器，以便将图像小部件相邻放置
# create a box container to place the image widget next to eachother
box_display = widgets.HBox([image_widget, Button_Close_Camera])
display(box_display)